---
# Tutorial Task: Sentiment-Based Quant Strategy
---

In this tutorial, you will design and implement a **quantitative trading strategy** that makes trading decisions based on **news sentiment** for a selected group of 3 stocks.

## Objective

Each day, over a 1-week historical period:
- **Buy** (go long) the stock with the **most positive** news sentiment
- **Sell** (go short) the stock with the **most negative** news sentiment

You will:
1. Select your own **universe of 3 stocks** (e.g., AAPL, BRK-B, DNUT, MSFT, GOOGL, AMZN, TSLA).
2. Use the provided News API to retrieve articles for each stock.
3. Calculate the **average sentiment** for each stock daily using tools like `TextBlob`.
4. Track your portfolio’s value over time by simulating simple returns.
5. Plot your portfolio value and analyze strategy performance.

## Constraints

- The News API is limited to **100 requests per day**. Avoid excessive API calls while developing by storing data or limiting requests.
- You only trade **one long and one short position** per day.
- Keep it simple: assume equal capital allocation to long and short trades.


## Step 1:
Import relevant libraries

In [ ]:
# Data analysis
import pandas as pd
import numpy as np

# API call
import requests

# Sentiment analysis
from textblob import TextBlob

# Time arithmetic
from datetime import datetime, timedelta

# Plotting
import matplotlib.pyplot as plt

# Financial data
import yfinance as yf


## Step 2:
Configure your API

https://newsapi.org/

In [ ]:
# Information to call API
API_KEY = '49473ca1043f406faa5dd31934688163' # Replace with your actual key
BASE_URL = 'https://newsapi.org/v2/everything'

## Step 3:
Create a function to get news and calculate the sentiment

In [ ]:
# We use TextBlob() to calculate sentiment:

# Show students how this works
print(TextBlob("This stock is amazing!").sentiment.polarity) # ~0.7-1 is a positive sentiment
print(TextBlob("This stock is okay").sentiment.polarity) # ~0.2 is a slightly positive sentiment
print(TextBlob("This stock is awful!").sentiment.polarity) # ~-0.8-1 is a negative sentiment
print(TextBlob("This is a stock").sentiment.polarity) # 0 is a neutral sentiment

In [ ]:
# We make a function to call the API and get the sentiment of a stock over a day

def get_sentiment(stock, date):
    from_date = date.strftime('%Y-%m-%d')
    to_date = (date + timedelta(days = 1)).strftime('%Y-%m-%d')

    params = {
        'q': stock,
        'from': from_date,
        'to': to_date,
        'sortBy': 'relevancy',
        'apiKey': API_KEY,
        'language': 'en',
        'pageSize': 5
    }

    # Call API with the relevant stock and dates
    response = requests.get(BASE_URL, params=params)
    data = response.json() # response in a .JSON format

    print(data)

    sentiments = []
    for article in data.get('articles', []):
        title = article.get('title', '')
        description = article.get('description', '')

        if not title and not description:
            continue #skip if both are missing
               
        text = f"{title} {description}"
        # Get sentiment of text
        sentiment = TextBlob(text).sentiment.polarity
        sentiments.append(sentiment)

    if sentiments:
        mean = np.mean(sentiments)
        print(mean)
        return mean # return the mean sentiment of the stock over the day
    return None


In [ ]:
get_sentiment('AAPL', datetime(2025, 5, 5))

## Step 4:
Simulate Trading by Backtesting

In [ ]:
def gather_daily_sentiments(stocks, start_date, end_date):
    """
    For each date in [start_date, end_date], calls get_sentiment(stock, date)
    and returns a DataFrame of shape (n_dates * n_stocks) of daily polarity
    """

    dates = pd.date_range(start_date, end_date)
    # initialise and empy dataframe
    sentiments = pd.DataFrame(index=dates, columns=stocks, dtype=float)

    for date in dates:
        for stock in stocks:
            sentiments.at[date, stock] = get_sentiment(stock, date)
    return sentiments

def fetch_price_data(stocks, start_date, end_date):
    """
    Downloads daily OHLC data for each ticker in 'stocks' between start_date and end_date. Returns a dict mapping ticker to its dataframe
    """

    price_data = {}
    for stock in stocks:
        df = yf.download(
            stock,
            start = start_date.strftime('%Y-%m-%d'),
            end = (end_date + timedelta(days = 1)).strftime('%Y-%m-%d'),
            interval = '1d',
            progress = False
        )
        price_data[stock] = df[['Open', 'Close']] # keep only what is needed
    return price_data

In [ ]:

# Define parameters for the API and its functions

START_DATE = pd.to_datetime('2025-04-28')
END_DATE = pd.to_datetime('2025-05-09')
STOCK_UNIVERSE = ['AAPL', 'DNUT', 'BRK-B']

#1) Get your sentiment surface

sentiment_df = gather_daily_sentiments(STOCK_UNIVERSE, START_DATE, END_DATE)

# 2) Pull in all your price data

price_dict = fetch_price_data(STOCK_UNIVERSE, START_DATE, END_DATE)

# Now you will have:
# - sentiment_df.loc[date, stock] which is polarity of the stock on that date
# - price_dict[stock].loc[date, ['Open', 'Close']] prices

In [ ]:

# Daily sentiment per stock

sentiment_df

In [ ]:
# Price data for each stock

print(price_dict['DNUT'])

In [ ]:
# Import backtesting functionality from python file

from momentum_functions import backtest_sentiment_strategy

In [ ]:
# Example usage

results = backtest_sentiment_strategy(sentiment_df, price_dict, init_capital=100000)
results


## Step 5:
Display results in ```matplotlib```

In [ ]:

# Plot the portfolio value

plt.figure(figsize=(10, 5))
plt.plot(results.index, results['portfolio_value'], marker = 'o', linewidth = 2)
plt.title('Sentiment-based strategy: portfolio value over two weeks')
plt.xlabel('Date')
plt.ylabel('Portfolio Value (USD)')
plt.axhline(y=100000, color='r', linestyle='--', label='Initial Capital')
plt.legend()
plt.grid(True)
plt.tight_layout()